In [11]:
import os
import sys
import platform
sys.path.append(os.path.join(os.getcwd().replace("notebooks/distilled_models", "")))

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from config import Config
from data.dataset import NetworkDataset, load_datasets
from model_config import MLP_Models, LSTM_Models, GRU_Models, CNN_models
from utils.benchmark import Benchmark

In [12]:
conf = Config()
load_model = GRU_Models()
model_conf = load_model.light_gru_4
model = load_model.get_model(model_conf)
model.load()
print(conf.device)

Checkpoint loaded from /global/D1/homes/jorgetf/Network-Packet-ML-Model/checkpoint/compressed_model/light_gru_4.pth!
cpu


In [13]:
# data loader
X_train, y_train, X_val, y_val, X_test, y_test = load_datasets(conf.datasets, model_type=load_model.type)
X_test, y_test = X_test[:500*conf.batch_size], y_test[:500*conf.batch_size]
print(X_test.shape)

dataset = NetworkDataset(X_test, y_test)
loader = DataLoader(dataset, conf.batch_size, shuffle=True)

torch.Size([500, 513, 1])


In [14]:
data, label = next(iter(loader))
print(data.shape, label.shape)

torch.Size([1, 513, 1]) torch.Size([1])


In [15]:
# benchmark model
name = model_conf["name"]
result_path = os.path.join(conf.benchmark_host, "compressed_model", name + ".txt")
benchmark = Benchmark(model, loader, conf.batch_size, name, result_path)
benchmark()

# print and save result
benchmark.print_result()
benchmark.save()

Benchmark - light_gru_4 model:

Memory usage (MB):
Avg memory usage: 1.742MB
Peak memory usage: 30.062MB

Model inference latency on one batch (batch size = 1):
Avg latency: 62.845ms
Min latency: 61.937ms
Max latency: 70.100ms

Model inference throughput (batch size = 1):
Throughput: 15.95 samples/sec

Model inference CPU usage (number of logical cores) during runtime:
CPU runtime: 3.14 seconds
Average CPU usage: 36.75/96 cores

Model (light_gru_4) Macro-F1, Micro-F1 and Macro ROC AUC scores:
Macro-F1 score: 0.45
Micro-F1 score: 0.95
Macro ROC AUC score: nan





/global/D1/homes/jorgetf/Network-Packet-ML-Model/.venv/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1201: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/global/D1/homes/jorgetf/Network-Packet-ML-Model/.venv/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1201: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/global/D1/homes/jorgetf/Network-Packet-ML-Model/.venv/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1201: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/global/D1/homes/jorgetf/Network-Packet-ML-Model/.venv/lib/python3.10/site-packages/sklearn/metrics/_ranking.py:1201: UndefinedMetricWarning: No positive samples in y_true, true positive value should be meaningless
  warnings.warn(
/global/D1/homes/jorgetf/Network-Packet-ML-Model/.venv/lib/python3.10/si